# All pairs shortest path algorithm

Shortest paths between every pair of vertices -algorithm implemented by QUBO in quantum annealer. Classical version of this is Floyd-Warshall algorithm: https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm

Here directed graph $G=(V,E)$ has sets of vertices $V$ and edges $E \subseteq \{(x,y)|(x,y) \in V^2 and x \not = y\}$. For every edge there is weight $w_{xy}$. Task is to find path with minimum sum of weights for every combination of $(s,t) \in E^2$.

In this solution QUBO has seperate elements for starting vertices $s \in V$ and traget vertices $t \in V$. In addtion it has one element for each edge $e \in E$. So alltogother QUBO is matrix of $(|V|+|V|+|E|) \times (|V|+|V|+|E|)$. First block is |V| elements which indicate what is the starting vertice $s$. Second block is |V| elements which indicate what is the terminating vertice $t$. Last block of $|E|$ indicates which edges form the shortest path between $s$ and $t$.

Penalty $p=\sum w_{xy}$.

Following constraints are build to QUBO: 
1. Exactly one s vertice (if more than one: $2p$)
2. Exactly one t vertice (if more than one: $2p$)
3. Vertices s and t are different (if not: $p$)
4. There should be one vertice starting from $s$ ($-p$ for each), no vertice should end to $s$ ($p$ for each)
5. There should be one vertice ending to $t$ ($-p$ for each), no vertice should start from $t$ ($p$ for each)
6. Two edges should not start/end to the same vertice, for example $s$ or $t$ (if so $2p$) 
7. Two edges should form a chain (ok chain gives $p$, otherwise more than that)
8. Pathing having lower weights should be prioritised

Proper path with correct $s$ and $t$ would give energy level of $w-p$ (constraint 7 brings minimum $p$ and constraint 4 and 5 both minimum $-p$). Because $p=\sum w_{xy}$ all samples with energy level below zero are correct paths. From this set of samples we choose lowest energy level sample for each $(s,t)$. 

In [1]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector

## Define graph

Input graph is array of tuples (1st vertice, 2nd vertice, weight)

In [2]:
E = np.array([(0, 2, 1), (2, 1, 2), (1, 3, 3), (3, 2, 4), (0, 1, 5)])
vertices = 4

Above graph visualised:

![](graph1.png)

In [3]:
#E = np.array([(0, 1, 8), (1, 3, 5), (0, 2, 2), (2, 1, 4), (2, 4, 7), (4, 3, 3)])
#vertices = 5

Above graph visualised:
    
![](graph2.png)

In [4]:
#E = np.array([(0, 1, 5), (0, 2, 5), (1, 0, 5), (1, 2, 2), (1, 3, 2),
#                 (2, 0, 5), (2, 1, 2), (2, 3, 10), (3, 1, 2), (3, 2, 10)])
#vertices = 4

Above graph visualised (every edge is biderctional):
    
![](graph3.png)

## Take max_count

In [5]:
max_count = 0
edges = len(E)
for e in E:
    max_count += e[2]
print('Max count:',max_count)

Max count: 15


## Some helper functions for getting results from sampleset

In [15]:
def path_from_sample(sample):
    s = 0
    t = 0
    w = 0
    for v in range(vertices):
        if sample['s'+str(v)]==1:
            s = v
        if sample['t'+str(v)]==1:
            t = v
    i = s
    path = str(i)
    while i!=t:
        for e in E:
            if e[0]==i and sample[str(e[0]) + '-' + str(e[1])]==1:
                i = e[1]
                path += '-'+str(i)
                w += e[2]
    return (str(s)+'-'+str(t),path,w)
        

def result_info(sampleset):
    ss = sampleset.filter(lambda s: s.energy<0)
    res = {}
    for s in ss:
        st, path, w = path_from_sample(s)
        if st not in res:
            res[st]=(path,w)
    return res

## Create QUBO from edge array

Constraints: 
1. Exactly one s vertice (if more than one: $2p$)
2. Exactly one t vertice (if more than one: $2p$)
3. Vertices s and t are different (if not: $p$)
4. There should be one vertice starting from $s$ ($-p$ for each), no vertice should end to $s$ ($p$ for each)
5. There should be one vertice ending to $t$ ($-p$ for each), no vertice should start from $t$ ($p$ for each)
6. Two edges should not start/end to the same vertice, for example $s$ or $t$ (if so $2p$) 
7. Two edges should form a chain (ok chain gives $p$, otherwise more than that)
8. Pathing having lower weights should be prioritised


In [25]:
Q = np.zeros((2*vertices + edges, 2*vertices + edges))
p = max_count

t1 = time.time()

# Constraints 1 and 2
for i in range(vertices):
    for j in range(i+1,vertices):
        Q[i,j]=2*p
        Q[vertices+i,j+vertices] = 2*p
        
# Constraint 3
for i in range(vertices):
    Q[i,i+vertices] = p

# Constraint 4
for v in range(vertices):
    for i,e in enumerate(E):
        if e[0]==v:
            Q[v,vertices*2+i] = -p
        if e[1]==v:
            Q[v,vertices*2+i] = p

# Constraint 5
for v in range(vertices):
    for i,e in enumerate(E):
        if e[1]==v:
            Q[vertices+v,vertices*2+i] = -p
        if e[0]==v:
            Q[vertices+v,vertices*2+i] = p

# Constraint 6
for i in range(edges):
    for j in range(i+1,edges):
        if E[i][0]==E[j][0] or E[i][1]==E[j][1]:
            Q[vertices*2+i,vertices*2+j] = 2*p

# Constraint 7
for i in range(edges):
    Q[vertices*2+i,vertices*2+i] += p
    for j in range(i+1,edges):
        if E[i][1]==E[j][0] or E[i][0]==E[j][1]:
            Q[vertices*2+i,vertices*2+j] += -p

# Constraint 8 
for i in range(edges):
    Q[vertices*2+i,vertices*2+i] += E[i][2]
    
qubo_time = (time.time()-t1)*1000
print('Time used for construction Q (ms): {:.3f}\n'.format(qubo_time))
print(Q)

Time used for construction Q (ms): 0.866

[[  0.  30.  30.  30.  15.   0.   0.   0. -15.   0.   0.   0. -15.]
 [  0.   0.  30.  30.   0.  15.   0.   0.   0.  15. -15.   0.  15.]
 [  0.   0.   0.  30.   0.   0.  15.   0.  15. -15.   0.  15.   0.]
 [  0.   0.   0.   0.   0.   0.   0.  15.   0.   0.  15. -15.   0.]
 [  0.   0.   0.   0.   0.  30.  30.  30.  15.   0.   0.   0.  15.]
 [  0.   0.   0.   0.   0.   0.  30.  30.   0. -15.  15.   0. -15.]
 [  0.   0.   0.   0.   0.   0.   0.  30. -15.  15.   0. -15.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -15.  15.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.  16. -15.   0.  30.  30.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.  17. -15. -15.  30.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  18. -15. -15.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  19.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  20.]]


In [18]:
labels = {}
for i in range(vertices):
    labels[i]='s'+str(i)
    labels[vertices+i]='t'+str(i)   
for i,e in enumerate(E):
    labels[vertices*2+i] = str(e[0]) + '-' + str(e[1])

## Create BQM from QUBO

In [26]:
t1 = time.time()
bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
bqm_time = (time.time()-t1)*1000
bqm = bqm.relabel_variables(labels, inplace=False)

## Local deterministic solver

In [28]:
t1 = time.time()
sampleset = dimod.ExactSolver().sample(bqm)
det_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}\n'.format(det_time))
print(sampleset.filter(lambda s: s.energy<0))

Time used (ms): 21.760

   0-1 0-2 1-3 2-1 3-2 s0 s1 s2 s3 t0 t1 t2 t3 energy num_oc.
0    0   1   0   0   0  1  0  0  0  0  0  1  0  -14.0       1
2    0   0   0   1   0  0  0  1  0  0  1  0  0  -13.0       1
1    0   1   0   1   0  1  0  0  0  0  1  0  0  -12.0       1
5    0   0   1   0   0  0  1  0  0  0  0  0  1  -12.0       1
8    0   0   0   0   1  0  0  0  1  0  0  1  0  -11.0       1
3    0   0   1   1   0  0  0  1  0  0  0  0  1  -10.0       1
11   1   0   0   0   0  1  0  0  0  0  1  0  0  -10.0       1
4    0   1   1   1   0  1  0  0  0  0  0  0  1   -9.0       1
7    0   0   0   1   1  0  0  0  1  0  1  0  0   -9.0       1
6    0   0   1   0   1  0  1  0  0  0  0  1  0   -8.0       1
10   1   0   1   0   0  1  0  0  0  0  0  0  1   -7.0       1
9    1   0   1   0   1  1  0  0  0  0  0  1  0   -3.0       1
['BINARY', 12 rows, 12 samples, 13 variables]


In [29]:
for k,v in result_info(sampleset).items():
    print('Route '+k+': '+v[0]+', weight '+str(v[1]))

Route 0-2: 0-2, weight 1
Route 2-1: 2-1, weight 2
Route 0-1: 0-2-1, weight 3
Route 1-3: 1-3, weight 3
Route 3-2: 3-2, weight 4
Route 2-3: 2-1-3, weight 5
Route 0-3: 0-2-1-3, weight 6
Route 3-1: 3-2-1, weight 6
Route 1-2: 1-3-2, weight 7


## Local heuristic classical solver

In [30]:
t1 = time.time()
sampleset2 = SimulatedAnnealingSampler().sample(bqm, num_reads=500).aggregate()
heur_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}\n'.format(heur_time))
print(sampleset2.filter(lambda s: s.energy<0))

Time used (ms): 125.934

   0-1 0-2 1-3 2-1 3-2 s0 s1 s2 s3 t0 t1 t2 t3 energy num_oc.
7    0   1   0   0   0  1  0  0  0  0  0  1  0  -14.0     126
4    0   0   0   1   0  0  0  1  0  0  1  0  0  -13.0      84
2    0   0   1   0   0  0  1  0  0  0  0  0  1  -12.0      61
5    0   1   0   1   0  1  0  0  0  0  1  0  0  -12.0      73
1    0   0   0   0   1  0  0  0  1  0  0  1  0  -11.0      46
3    1   0   0   0   0  1  0  0  0  0  1  0  0  -10.0      33
9    0   0   1   1   0  0  0  1  0  0  0  0  1  -10.0      22
0    0   1   1   1   0  1  0  0  0  0  0  0  1   -9.0      18
8    0   0   0   1   1  0  0  0  1  0  1  0  0   -9.0      16
10   0   0   1   0   1  0  1  0  0  0  0  1  0   -8.0       9
6    1   0   1   0   0  1  0  0  0  0  0  0  1   -7.0      12
['BINARY', 11 rows, 500 samples, 13 variables]


In [31]:
for k,v in result_info(sampleset2).items():
    print('Route '+k+': '+v[0]+', weight '+str(v[1]))

Route 0-2: 0-2, weight 1
Route 2-1: 2-1, weight 2
Route 1-3: 1-3, weight 3
Route 0-1: 0-2-1, weight 3
Route 3-2: 3-2, weight 4
Route 2-3: 2-1-3, weight 5
Route 0-3: 0-2-1-3, weight 6
Route 3-1: 3-2-1, weight 6
Route 1-2: 1-3-2, weight 7


## Quantum solver

In [32]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [33]:
sampleset3 = EmbeddingComposite(machine).sample(bqm, num_reads=500)

In [34]:
qpu_time = sampleset3.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset3.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Physical qubits used: {}\n'.format(qubits))
print(sampleset3.filter(lambda s: s.energy<0))

QPU time used (ms): 74.3
Physical qubits used: 22

   0-1 0-2 1-3 2-1 3-2 s0 s1 s2 s3 t0 t1 t2 t3 energy num_oc. chain_.
0    0   1   0   0   0  1  0  0  0  0  0  1  0  -14.0      43     0.0
1    0   0   0   1   0  0  0  1  0  0  1  0  0  -13.0      61     0.0
2    0   0   1   0   0  0  1  0  0  0  0  0  1  -12.0      23     0.0
3    0   1   0   1   0  1  0  0  0  0  1  0  0  -12.0      54     0.0
4    0   0   0   0   1  0  0  0  1  0  0  1  0  -11.0      17     0.0
5    0   0   1   1   0  0  0  1  0  0  0  0  1  -10.0      21     0.0
6    1   0   0   0   0  1  0  0  0  0  1  0  0  -10.0      49     0.0
7    0   1   1   1   0  1  0  0  0  0  0  0  1   -9.0      21     0.0
8    0   0   0   1   1  0  0  0  1  0  1  0  0   -9.0      23     0.0
9    0   0   1   0   1  0  1  0  0  0  0  1  0   -8.0      10     0.0
10   1   0   1   0   0  1  0  0  0  0  0  0  1   -7.0      22     0.0
11   1   0   1   0   1  1  0  0  0  0  0  1  0   -3.0       8     0.0
['BINARY', 12 rows, 352 samples, 13 var

In [35]:
for k,v in result_info(sampleset3).items():
    print('Route '+k+': '+v[0]+', weight '+str(v[1]))

Route 0-2: 0-2, weight 1
Route 2-1: 2-1, weight 2
Route 1-3: 1-3, weight 3
Route 0-1: 0-2-1, weight 3
Route 3-2: 3-2, weight 4
Route 2-3: 2-1-3, weight 5
Route 0-3: 0-2-1-3, weight 6
Route 3-1: 3-2-1, weight 6
Route 1-2: 1-3-2, weight 7


In [36]:
dwave.inspector.show(sampleset3)

Serving Inspector on http://127.0.0.1:18000/?problemId=b1edf498-7f96-4669-92f3-4d4f7e6c10b4

'http://127.0.0.1:18000/?problemId=b1edf498-7f96-4669-92f3-4d4f7e6c10b4'

## Hybrid solver

Hybrid solver nrings only best solution, not good in here

In [45]:
#sampleset4 = LeapHybridSampler().sample(bqm)

In [46]:
#hyb_time = sampleset4.info['qpu_access_time'] / 1000
#print('QPU time used (ms): {:.1f}\n'.format(hyb_time))
#print(sampleset4.filter(lambda s: s.energy<0))

In [47]:
#for k,v in result_info(sampleset4).items():
#    print('Route '+k+': '+v[0]+', weight '+str(v[1]))

## Timings

In [49]:
print('Construting QUBO: {:.3f}'.format(qubo_time))
print('Construting BQM: {:.3f}'.format(bqm_time))
print('\nLocal deterministic solver: {:.1f}'.format(det_time))
print('Local heuristic solver: {:.1f}'.format(heur_time))
print('Quantum solver: {:.1f}'.format(qpu_time))

Construting QUBO: 0.866
Construting BQM: 0.288

Local deterministic solver: 21.8
Local heuristic solver: 125.9
Quantum solver: 74.3
